In [1]:
from transformers import AutoTokenizer, AutoModel

In [2]:
mod = AutoModel.from_pretrained('Qwen/Qwen2.5-0.5B')

In [3]:
for n, p in mod.named_parameters():
    print(n, p.shape)

embed_tokens.weight torch.Size([151936, 896])
layers.0.self_attn.q_proj.weight torch.Size([896, 896])
layers.0.self_attn.q_proj.bias torch.Size([896])
layers.0.self_attn.k_proj.weight torch.Size([128, 896])
layers.0.self_attn.k_proj.bias torch.Size([128])
layers.0.self_attn.v_proj.weight torch.Size([128, 896])
layers.0.self_attn.v_proj.bias torch.Size([128])
layers.0.self_attn.o_proj.weight torch.Size([896, 896])
layers.0.mlp.gate_proj.weight torch.Size([4864, 896])
layers.0.mlp.up_proj.weight torch.Size([4864, 896])
layers.0.mlp.down_proj.weight torch.Size([896, 4864])
layers.0.input_layernorm.weight torch.Size([896])
layers.0.post_attention_layernorm.weight torch.Size([896])
layers.1.self_attn.q_proj.weight torch.Size([896, 896])
layers.1.self_attn.q_proj.bias torch.Size([896])
layers.1.self_attn.k_proj.weight torch.Size([128, 896])
layers.1.self_attn.k_proj.bias torch.Size([128])
layers.1.self_attn.v_proj.weight torch.Size([128, 896])
layers.1.self_attn.v_proj.bias torch.Size([128])

- 24 layers
- 151936 input tokens
- 869: size of each vectors. Hidden size of model

## Function calling: Ollama

In [10]:
import ollama
import requests
import json

messages = [{'role': 'user', 'content': 'What is the main news right now in the USA?'}]

def getnews(c):
    c=c.lower().strip()
    if c=='france': s='fr'
    elif c=='india': s='in'
    elif c=='usa': s='us'
    elif c=='australia': s='au'
    elif c=='russia': s='ru'
    elif c=='united kingdom': s='gb'
    else:
        print("unknown country",c)
        s='fr'
    url="https://saurav.tech/NewsAPI/top-headlines/category/general/"+s+".json"
    print("calling fct")
    response = requests.get(url)
    res = response.text
    print("tool res",res)
    print("\n"*5)

    n=json.loads(res)
    r=n['articles'][0]['title']+": "+n['articles'][0]['content']
    print("extracting news",r,"\n"*3)
    return r

def main():
    response = ollama.chat(
        model=mod,
        messages=messages,
        tools=[
          {
            'type': 'function',
            'function': {
              'name': 'getnews',
              'description': 'Get recent news from a country',
              'parameters': {
                'type': 'object',
                'properties': {
                    'country': {
                        'type': 'string',
                        'description': 'The name of the country',
                        },
                },
                'required': ['country'],
              },
            },
          },
        ],
    )

    # Add the model's response to the conversation history
    messages.append(response['message'])
    print("first answer",response['message'])

    # Check if the model decided to use the provided function
    if not response['message'].get('tool_calls'):
        print("The model didn't use the function. Its response was:")
        print(response['message']['content'])
        return

    # Process function calls made by the model
    if response['message'].get('tool_calls'):
        available_functions = {
          'getnews': getnews,
        }
        for tool in response['message']['tool_calls']:
          function_to_call = available_functions[tool['function']['name']]
          function_response = function_to_call(tool['function']['arguments']['country'])
          # Add function response to the conversation
          messages.append(
            {
              'role': 'tool',
              'content': function_response,
            }
          )

    # Second API call: Get final response from the model
    final_response = ollama.chat(model=mod, messages=messages)
    print(final_response['message']['content'])

main()

TypeError: Object of type Qwen2Model is not JSON serializable